In [1]:
import ebooklib
from ebooklib import epub
import re
import os

import enchant

import gensim
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import words, stopwords, names

import nltk
nltk.download('wordnet')
nltk.download('stopwords')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

D:\DesignAPP\anaconda3\envs\demoEnvpy38\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\46981\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\46981\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def merge_strings_until_limit(strings, min_length, max_length, test_for_max = 0):
    merged_string = ""
    merged_strings = []

    for s in strings:
        if len(merged_string) <= min_length:# and len(s)<max_length:
            merged_string += s
        
        elif len(merged_string) > max_length and test_for_max<5:
            splitParagraph = merged_string.split('.')
            splitParagraphRePoint = []
            for sp in splitParagraph:
                splitParagraphRePoint.append(sp+'.')
            merged = merge_strings_until_limit(splitParagraphRePoint, min_length, max_length, test_for_max+1)
            merged_strings.extend(merged)
            merged_string = s
        else:
            merged_strings.append(merged_string)
            merged_string = s

    if merged_string:
        merged_strings.append(merged_string)

    return merged_strings

In [3]:
def read_epub_paragraphs(epub_file, ID):
    book = epub.read_epub(epub_file)
    paragraphs = []

    for item in book.get_items():
        if item.get_type() == ebooklib.ITEM_DOCUMENT:
            content = item.get_content().decode('utf-8')
            content = re.sub('<[^<]+?>', '', content)  # Remove HTML tags
            content = re.sub('\s+', ' ', content)  # Replace multiple whitespaces with a single space
            content = re.sub('\n', ' ', content)
            paragraphs.extend(content.strip().split("&#13;"))
    paragraphs = merge_strings_until_limit(paragraphs, 200, 1000)
    paragraphs = [{'paragraph':paragraphs[i], 'nr':i,'bookID':ID} for i in range(len(paragraphs)) if len(paragraphs[i])>50]
    return paragraphs[1:-1]

In [4]:
read_epub_paragraphs("Flipped.epub",1)

D:\DesignAPP\anaconda3\envs\demoEnvpy38\lib\site-packages\ebooklib\epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


[{'paragraph': 'Dedicated with infinite love toColton and Connor,who make me feel like so much more than the sum of my parts.Special thanks to …my husband, Mark Parsons,who helps me feel the magic,andmy excellent editor, Nancy Siscoe,for her care and insight(and for making me stick to a reduced-filler diet). Also, eternal gratitude toTad Callahan and Patricia Gabel,who were on the ball when we needed it most. Finally, thanks to Jeanne Madrid and the staff at Casa De Vida—may you keep the spirit.',
  'nr': 1,
  'bookID': 1},
 {'paragraph': 'CONTENTS Diving Under Flipped Buddy, Beware! The Sycamore Tree Brawk-Brawk-Brawk! The Eggs Get a Grip, Man The Yard Looming Large and Smelly The Visit The Serious Willies The Dinner Flipped The Basket Boys',
  'nr': 2,
  'bookID': 1},
 {'paragraph': "Diving Under All I've ever wanted is for Juli Baker to leave me alone. For her to back off — you know, just give me some space. It all started the summer before second grade when our moving van pulled in

# Vectorise Text - 1

In [5]:
ENGLISH_DICT1 = enchant.Dict("en_UK")
ENGLISH_DICT2 = enchant.Dict("en_US")
def is_english_word(word):
    return (ENGLISH_DICT1.check(word) or ENGLISH_DICT2.check(word))


In [6]:
paragraphs = read_epub_paragraphs("Flipped.epub",1)

In [7]:
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess(paragraphs):
    processed_paragraphs = []

    for paragraph in paragraphs:
        words = gensim.utils.simple_preprocess(paragraph, min_len=3, deacc=True)

        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

        filtered_words = [word for word in lemmatized_words if (word not in stop_words) and (is_english_word(word))]

        stemmed_words = [stemmer.stem(word) for word in filtered_words]

        processed_paragraph = " ".join(stemmed_words)
        processed_paragraphs.append(processed_paragraph)

    return processed_paragraphs

In [8]:
processedDoc = preprocess([p['paragraph'] for p in paragraphs])
print(processedDoc)

['dedic infinit love make feel like much sum part special thank husband mark parson help feel magic excel editor nanci care insight make stick reduc filler diet also etern gratitud ball need final thank staff casa may keep spirit', 'content dive flip buddi bewar sycamor tree egg get grip man yard loom larg smelli visit seriou willi dinner flip basket boy', 'dive ever want baker leav alon back know give space start summer second grade move van pull neighborhood', 'sinc done eighth grade friend make half decad strateg avoid social discomfort barg life barg shove wedg way life', 'invit get move van start climb box exactli take show like baker dad tri stop hey say catapult board', 'get mud everywher true shoe like cake stuff hop though instead plant rear end floor start push big box foot', 'want help glanc way sure look like need like implic even though dad toss sort look week could tell like girl either', 'hey warn realli valuabl thing box well one scoot box label look way push togeth dad

In [9]:
vectorizer = TfidfVectorizer(min_df=3)
tfidf_matrix = vectorizer.fit_transform(processedDoc)

In [10]:
query = 'love is a tree in the eden garden'
processedQuery = preprocess([query])[0]
print(processedQuery)

love tree garden


In [11]:
query_vector = vectorizer.transform([processedQuery])
similarities = cosine_similarity(query_vector, tfidf_matrix)

nearest_neighbor_index = similarities.argmax()
nearest_neighbor = tfidf_matrix[nearest_neighbor_index]
similarity_score = similarities[0, nearest_neighbor_index]

print(nearest_neighbor_index, similarity_score)

137 0.520119437996897


In [12]:
paragraphs[137]

{'paragraph': " The girl arguing with those men was Juli — the world's peskiest, bossiest, most know-it-all female. But all of a sudden my stomach completely bailed on me. Juli loved that tree. Stupid as it was, she loved that tree, and cutting it down would be like cutting out her heart.",
 'nr': 139,
 'bookID': 1}

### apply SVD for dimensionality reduction

In [13]:
import numpy as np
from scipy.sparse import random
from sklearn.decomposition import TruncatedSVD

In [17]:
n_components = 250  # Number of components to keep after reduction
svd = TruncatedSVD(n_components=n_components, algorithm='randomized')
reduced_matrix = svd.fit_transform(tfidf_matrix)
reduced_query_vec = svd.transform(query_vector)

similarities2 = cosine_similarity(reduced_query_vec, reduced_matrix)
nearest_neighbor_index = similarities2.argmax()
nearest_neighbor = tfidf_matrix[nearest_neighbor_index]
similarity_score = similarities2[0, nearest_neighbor_index]
print(nearest_neighbor_index)

137


# Vectorise Text - 2

In [20]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [21]:
def preprocess2(paragraphs):
    processed_paragraphs = []

    for paragraph in paragraphs:
        words = gensim.utils.simple_preprocess(paragraph, min_len=3, deacc=True)

        lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

        filtered_words = [word for word in lemmatized_words if (word not in stop_words) and (is_english_word(word))]

        processed_paragraph = " ".join(filtered_words)
        processed_paragraphs.append(processed_paragraph)

    return processed_paragraphs

In [22]:
texts = preprocess2([p['paragraph'] for p in paragraphs])

In [23]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(texts)]
model = Doc2Vec(vector_size=250, min_count=0, alpha = 0.025, min_alpha=0.025, epochs=100)
model.build_vocab(documents)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

In [24]:
query = 'love is a tree in the eden garden'
tokens = preprocess2([query])[0]
vector = model.infer_vector([tokens])
print(tokens)

love tree garden


In [25]:
model.dv.most_similar(positive=[vector])

[(697, 0.15824860334396362),
 (305, 0.14663472771644592),
 (283, 0.14531643688678741),
 (500, 0.13934168219566345),
 (280, 0.13453452289104462),
 (38, 0.13269253075122833),
 (292, 0.12806138396263123),
 (466, 0.12796489894390106),
 (783, 0.12703655660152435),
 (486, 0.12503522634506226)]

In [26]:
paragraphs[12]

{'paragraph': " I smiled and said, “Sure thing!” then jumped off the liftgate and headed for my new front door. I heard her coming after me but I couldn't believe it. Maybe it just sounded like she was chasing me; maybe she was really going the other way.",
 'nr': 13,
 'bookID': 1}

# Homework 4
Write a function that takes a sentence as an input and returns a dictionary with: 1. The best match from your text libary based on the TF-IDF vector 2. The best match from your text library based on the reduced SVD vector 3. The best match based on the Doc2Vec model (Hint: first make sure you create a well ordere data structure including the different vectors for your library of text)

In [18]:
def find_best_match(sentence):
    # Preprocess the input sentence
    processed_query = preprocess([sentence])[0]

    # TF-IDF vector
    query_vector_tfidf = vectorizer.transform([processed_query])
    similarities_tfidf = cosine_similarity(query_vector_tfidf, tfidf_matrix)
    best_match_tfidf_index = similarities_tfidf.argmax()
    best_match_tfidf_score = similarities_tfidf[0, best_match_tfidf_index]
    best_match_tfidf_paragraph = paragraphs[best_match_tfidf_index]

    # Reduced SVD vector
    query_vector_svd = svd.transform(query_vector_tfidf)
    similarities_svd = cosine_similarity(query_vector_svd, reduced_matrix)
    best_match_svd_index = similarities_svd.argmax()
    best_match_svd_score = similarities_svd[0, best_match_svd_index]
    best_match_svd_paragraph = paragraphs[best_match_svd_index]

    # Doc2Vec model
    query_vector_doc2vec = model.infer_vector([processed_query])
    most_similar_doc = model.dv.most_similar(positive=[query_vector_doc2vec])[0]
    best_match_doc2vec_index = most_similar_doc[0]
    best_match_doc2vec_score = most_similar_doc[1]
    best_match_doc2vec_paragraph = paragraphs[best_match_doc2vec_index]

    # Create and return the result dictionary
    result_dict = {
        "TF-IDF Best Match": {
            "Index": best_match_tfidf_index,
            "Score": best_match_tfidf_score,
            "Paragraph": best_match_tfidf_paragraph
        },
        "SVD Best Match": {
            "Index": best_match_svd_index,
            "Score": best_match_svd_score,
            "Paragraph": best_match_svd_paragraph
        },
        "Doc2Vec Best Match": {
            "Index": best_match_doc2vec_index,
            "Score": best_match_doc2vec_score,
            "Paragraph": best_match_doc2vec_paragraph
        }
    }

    return result_dict

In [28]:
input_sentence = 'love is a tree in the eden garden'
result = find_best_match(input_sentence)
print(result)

{'TF-IDF Best Match': {'Index': 137, 'Score': 0.520119437996897, 'Paragraph': {'paragraph': " The girl arguing with those men was Juli — the world's peskiest, bossiest, most know-it-all female. But all of a sudden my stomach completely bailed on me. Juli loved that tree. Stupid as it was, she loved that tree, and cutting it down would be like cutting out her heart.", 'nr': 139, 'bookID': 1}}, 'SVD Best Match': {'Index': 137, 'Score': 0.7109712671023787, 'Paragraph': {'paragraph': " The girl arguing with those men was Juli — the world's peskiest, bossiest, most know-it-all female. But all of a sudden my stomach completely bailed on me. Juli loved that tree. Stupid as it was, she loved that tree, and cutting it down would be like cutting out her heart.", 'nr': 139, 'bookID': 1}}, 'Doc2Vec Best Match': {'Index': 697, 'Score': 0.15824860334396362, 'Paragraph': {'paragraph': " “Well, so? It's not your fault her yard's a mess.” “But then I found out that they don't even own that house. They'

# Homework 5
Write a function that takes a sentence and a number K that gives back a list of K best matches in sorted order. (use the type of vectorisation explored in the previous exercise that you find most successful).

In [29]:
def find_k_best_matches_tfidf(sentence, k):
    # Preprocess the input sentence
    processed_query = preprocess([sentence])[0]

    # Vectorize the query using the TF-IDF vectorizer
    query_vector = vectorizer.transform([processed_query])

    # Calculate cosine similarities with all paragraphs
    similarities = cosine_similarity(query_vector, tfidf_matrix)

    # Find K best matches
    best_matches = []
    for _ in range(k):
        nearest_neighbor_index = similarities.argmax()
        similarity_score = similarities[0, nearest_neighbor_index]
        most_similar_paragraph = paragraphs[nearest_neighbor_index]

        # Append information to the result list
        best_matches.append({
            "Index": nearest_neighbor_index,
            "Score": similarity_score,
            "Paragraph": most_similar_paragraph
        })

        # Set the similarity to zero for the already selected paragraph
        similarities[0, nearest_neighbor_index] = 0

    # Sort the result list by similarity score in descending order
    best_matches = sorted(best_matches, key=lambda x: x["Score"], reverse=True)

    # Display results on separate lines
    for match in best_matches:
        print(f"Index: {match['Index']}, Score: {match['Score']}")
        print(match['Paragraph'])
        print("\n" + "-"*50 + "\n")

    return best_matches

In [30]:
input_sentence = 'love is a tree in the eden garden'
k = 10
result_tfidf = find_k_best_matches_tfidf(input_sentence, k)

Index: 137, Score: 0.520119437996897
{'paragraph': " The girl arguing with those men was Juli — the world's peskiest, bossiest, most know-it-all female. But all of a sudden my stomach completely bailed on me. Juli loved that tree. Stupid as it was, she loved that tree, and cutting it down would be like cutting out her heart.", 'nr': 139, 'bookID': 1}

--------------------------------------------------

Index: 320, Score: 0.4935613358067432
{'paragraph': " She's over the top about everything!” “Bryce, I asked you to conquer your fear, but all you did was give in to it. If you were in love with her, that would be one thing. Love is something to be afraid of, but this, this is embarrassing.", 'nr': 322, 'bookID': 1}

--------------------------------------------------

Index: 174, Score: 0.4467285054145193
{'paragraph': "The Sycamore Tree I love to watch my father paint. Or really, I love to hear him talk while he paints. The words always come out soft and somehow heavy when he's brushing 

# Homework 6
Using fuyu or clip write a function that takes an image and returns a list of most relevant sentences from your text libary.

### Cannot run locally, use Colab
https://colab.research.google.com/drive/18r6YFqWyUDLmYri99wx-UllBt7xferHW#scrollTo=XyDNFE9-ikQ4
